# 🎯 SQLAlchemy 연습문제 (Relationship 중심)

In [ ]:
# 문제 풀이용 기본 설정
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, DateTime, ForeignKey, func, and_, or_, desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

engine = create_engine('sqlite:///:memory:', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

session = Session()

# 모델 정의
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column(String)
    email = Column(String)
    age = Column(Integer)
    salary = Column(Float)
    is_active = Column(Boolean, default=True)

    orders = relationship("Order", back_populates="user")

    def __repr__(self):
        return f"<User({self.username}, {self.age}세)>"

class Product(Base):
    __tablename__ = "products"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    price = Column(Float)
    stock = Column(Integer)

    order_items = relationship("OrderItem", back_populates="product")

    def __repr__(self):
        return f"<Product({self.name}, {self.price}원)>"

class Order(Base):
    __tablename__ = "orders"
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id"))
    total_amount = Column(Float)
    status = Column(String)
    order_date = Column(DateTime, default=datetime.now)

    user = relationship("User", back_populates="orders")
    order_items = relationship("OrderItem", back_populates="order")

    def __repr__(self):
        return f"<Order({self.id}, {self.total_amount}원, {self.status})>"

class OrderItem(Base):
    __tablename__ = "order_items"
    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey("orders.id"))
    product_id = Column(Integer, ForeignKey("products.id"))
    quantity = Column(Integer)

    order = relationship("Order", back_populates="order_items")
    product = relationship("Product", back_populates="order_items")

Base.metadata.create_all(engine)

# 테스트 데이터 생성
def create_test_data():
    # 사용자 생성
    users = [
        User(username="김철수", email="kim@test.com", age=25, salary=50000, is_active=True),
        User(username="이영희", email="lee@test.com", age=30, salary=70000, is_active=True),
        User(username="박민수", email="park@test.com", age=28, salary=60000, is_active=False),
        User(username="최지원", email="choi@test.com", age=35, salary=80000, is_active=True),
        User(username="정다영", email="jung@test.com", age=26, salary=45000, is_active=True)
    ]

    # 제품 생성
    products = [
        Product(name="노트북", category="전자제품", price=1500000, stock=20),
        Product(name="마우스", category="전자제품", price=30000, stock=50),
        Product(name="키보드", category="전자제품", price=80000, stock=30),
        Product(name="티셔츠", category="의류", price=25000, stock=100),
        Product(name="청바지", category="의류", price=70000, stock=40),
        Product(name="운동화", category="신발", price=120000, stock=25)
    ]

    session.add_all(users + products)
    session.commit()

    # 주문 생성
    orders = [
        Order(user_id=1, total_amount=1610000, status="completed"),    # 김철수
        Order(user_id=1, total_amount=95000, status="shipped"),       # 김철수
        Order(user_id=2, total_amount=220000, status="completed"),    # 이영희
        Order(user_id=4, total_amount=150000, status="pending"),      # 최지원
        Order(user_id=5, total_amount=45000, status="cancelled")      # 정다영
    ]

    session.add_all(orders)
    session.commit()

    # 주문 항목 생성
    order_items = [
        # 김철수 첫 번째 주문 (노트북 + 마우스 + 키보드)
        OrderItem(order_id=1, product_id=1, quantity=1),  # 노트북
        OrderItem(order_id=1, product_id=2, quantity=1),  # 마우스
        OrderItem(order_id=1, product_id=3, quantity=1),  # 키보드

        # 김철수 두 번째 주문 (티셔츠 + 청바지)
        OrderItem(order_id=2, product_id=4, quantity=1),  # 티셔츠
        OrderItem(order_id=2, product_id=5, quantity=1),  # 청바지

        # 이영희 주문 (운동화 + 청바지)
        OrderItem(order_id=3, product_id=6, quantity=1),  # 운동화
        OrderItem(order_id=3, product_id=5, quantity=1),  # 청바지
        OrderItem(order_id=3, product_id=2, quantity=1),  # 마우스

        # 최지원 주문 (노트북)
        OrderItem(order_id=4, product_id=1, quantity=1),  # 노트북

        # 정다영 주문 (취소됨)
        OrderItem(order_id=5, product_id=4, quantity=1),  # 티셔츠
    ]

    session.add_all(order_items)
    session.commit()
    print("✅ 테스트 데이터 생성 완료!")

create_test_data()

## 📝 문제 1번 (기본 - Relationship 활용)

In [ ]:
"""
🎯 문제 1: 기본 Relationship 활용 (난이도: ⭐)

김철수 사용자의 모든 주문 정보를 다음과 같이 출력하세요:
- 사용자 이름: 김철수
- 총 주문 수: X개
- 각 주문별로:
  - 주문 ID: X
  - 주문 금액: X원
  - 주문 상태: X

힌트: user.orders를 활용하세요!
"""

def problem_1():
    """여기에 코드를 작성하세요"""
    # user = session.query(User).filter(User.username == "김철수").first()
    # ...
    pass

print("🎯 문제 1번 실행:")
problem_1()

## 📝 문제 2번 (집계함수 + Relationship)

In [ ]:
"""
🎯 문제 2: 집계함수와 Relationship (난이도: ⭐⭐)

각 사용자별로 다음 정보를 출력하세요:
- 사용자 이름
- 총 주문 횟수 (len 사용)
- 총 주문 금액의 합계 (sum 사용)
- 평균 주문 금액 (직접 계산)

단, 주문이 없는 사용자도 포함하되 "주문 없음"으로 표시하세요.

힌트: user.orders의 길이와 반복문을 활용하세요!
"""

def problem_2():
    """여기에 코드를 작성하세요"""
    # users = session.query(User).all()
    # for user in users:
    #     ...
    pass

print("\n🎯 문제 2번 실행:")
problem_2()

## 📝 문제 3번 (조건부 필터링 + 정렬)

In [ ]:
"""
🎯 문제 3: 조건부 필터링과 정렬 (난이도: ⭐⭐⭐)

다음 조건을 만족하는 제품들을 찾아 출력하세요:
1. 전자제품 카테고리
2. 재고가 30개 이상
3. 가격 순으로 정렬 (비싼 것부터)

각 제품에 대해 다음 정보를 출력:
- 제품명: X
- 가격: X원
- 재고: X개
- 이 제품을 주문한 사용자들: [사용자명1, 사용자명2, ...]

힌트: product.order_items에서 order.user로 접근!
"""

def problem_3():
    """여기에 코드를 작성하세요"""
    # products = session.query(Product).filter(
    #     and_(Product.category == "전자제품", Product.stock >= 30)
    # ).order_by(desc(Product.price)).all()
    # ...
    pass

print("\n🎯 문제 3번 실행:")
problem_3()

## 📝 문제 4번 (그룹화 + Relationship)

In [ ]:
"""
🎯 문제 4: 카테고리별 통계 (난이도: ⭐⭐⭐⭐)

각 제품 카테고리별로 다음 통계를 출력하세요:
1. 카테고리명
2. 카테고리 내 제품 수
3. 카테고리 내 평균 가격
4. 이 카테고리 제품들을 구매한 고유 사용자 수 (중복 제거)

힌트:
- group_by() 사용
- set()을 사용해서 중복 사용자 제거
- product.order_items를 통해 구매 사용자 접근
"""

def problem_4():
    """여기에 코드를 작성하세요"""
    # 카테고리별로 그룹화
    # category_groups = session.query(Product.category).distinct().all()
    # for (category,) in category_groups:
    #     products_in_category = session.query(Product).filter(Product.category == category).all()
    #     ...
    pass

print("\n🎯 문제 4번 실행:")
problem_4()

## 📝 문제 5번 (복합 조건 + 고급 활용)

In [ ]:
"""
🎯 문제 5: VIP 고객 분석 (난이도: ⭐⭐⭐⭐⭐)

다음 조건을 만족하는 "VIP 고객"들을 찾아 분석하세요:

VIP 조건:
1. 활성 사용자 (is_active = True)
2. 연봉이 60000 이상
3. 완료된 주문(completed)이 1개 이상 있음

각 VIP 고객에 대해 출력:
- 고객명: X
- 연봉: X원
- 완료된 주문 수: X개
- 완료된 주문 총액: X원
- 가장 비싼 단일 주문 금액: X원
- 주문한 제품 카테고리들: [카테고리1, 카테고리2, ...] (중복 제거)

마지막에 VIP 고객들을 완료된 주문 총액 순으로 정렬해서 출력하세요.

힌트:
- 여러 조건을 and_()로 결합
- order.status == "completed" 조건 확인
- set()으로 카테고리 중복 제거
- 리스트 정렬 사용
"""

def problem_5():
    """여기에 코드를 작성하세요"""
    # vip_candidates = session.query(User).filter(
    #     and_(User.is_active == True, User.salary >= 60000)
    # ).all()
    #
    # vip_customers = []
    # for user in vip_candidates:
    #     completed_orders = [order for order in user.orders if order.status == "completed"]
    #     if len(completed_orders) >= 1:
    #         ...
    pass

print("\n🎯 문제 5번 실행:")
problem_5()

## 💡 문제 풀이 팁

In [ ]:
# 🔑 핵심 패턴들

# 1. Relationship으로 관련 데이터 접근
user.orders          # 사용자 → 모든 주문들
order.user          # 주문 → 주문자
product.order_items # 제품 → 모든 주문 항목들
order_item.order    # 주문 항목 → 주문
order_item.product  # 주문 항목 → 제품

# 2. 리스트 컴프리헨션 활용
completed_orders = [order for order in user.orders if order.status == "completed"]

# 3. 중복 제거
unique_users = set()
unique_users.add(user.username)
result = list(unique_users)

# 4. 정렬
vip_list.sort(key=lambda x: x['total_amount'], reverse=True)  # 내림차순